<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#1-加载数据" data-toc-modified-id="1-加载数据-1">1 加载数据</a></span></li><li><span><a href="#2-封装为类" data-toc-modified-id="2-封装为类-2">2 封装为类</a></span></li><li><span><a href="#3-打印每辆车的行车路线" data-toc-modified-id="3-打印每辆车的行车路线-3">3 打印每辆车的行车路线</a></span></li><li><span><a href="#4-总结：" data-toc-modified-id="4-总结：-4">4 总结：</a></span></li></ul></div>

多辆车的路径规划 VRP：     
条件：经过中国33个城市，一共4辆车，每辆车最大行驶10000公里    
目标：使得每辆车的行驶里程数更接近     
需要注意：    
1）在VRP问题中，路径上给点赋的index和点实际的index不一样，需要使用IndexToNode方法进行转换才能得到实际的index    
完成带有约束条件的VRP问题


In [1]:
import pandas as pd
from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp

# 1 加载数据

In [2]:
df_distance = pd.read_excel("distance.xlsx", index_col=0)

In [3]:
df_distance.head(2)

,北京,天津,哈尔滨,长春,沈阳,石家庄,呼和浩特,太原,上海,杭州,...,成都,重庆,南宁,贵阳,昆明,武汉,长沙,南昌,香港,澳门
北京,0,122476,1229869,987941,682963,294125,491658,487353,1210971,1277042,...,1835076,1753571,2346302,2086066,2587861,1161356,1477547,1431143,2203897,2273144
天津,122476,0,1201742,959814,654836,314982,614846,508210,1093989,1217026,...,1855933,1774428,2343851,2083615,2585410,1158905,1475096,1358857,2201446,2270693


# 2 封装为类

In [4]:
class VRP:
    def __init__(self, df_distance, num_vehicles=1, depot=0, city_names=None):
        # 设置城市名称
        self.df = df_distance
        self.num_vehicles = num_vehicles
        self.depot = depot
        self.all_city = self.df.columns
        if city_names is not None:
            self.city_names = city_names
            self.df = self.df.loc[self.all_city.isin(city_names),
                                  self.city_names]
        else:
            self.city_names = self.all_city

    def get_solution(self, manager, routing, solution):
        # 记录每辆车的里程
        distance_list = []
        # 记录每辆车的路径规划
        route_list = []

        for vehicle_id in range(self.num_vehicles):
            route = []
            # 从vehicle_id的起始节点出发
            index = routing.Start(vehicle_id)
            # 记录车最终行驶的距离
            route_distance = 0
            # routing.IsEnd 判断路径是否结束
            while not routing.IsEnd(index):
                # IndexToNode将manager中的index转换为distance_matrix中的index
                name_index = manager.IndexToNode(index)
                route.append(name_index)
                previous_index = index
                index = solution.Value(routing.NextVar(index))
                # 针对vehicle=0，统计从previous_index到index节点的距离
                route_distance += routing.GetArcCostForVehicle(
                    previous_index, index, 0)
            route_list.append(route)
            distance_list.append(route_distance)
        return route_list, distance_list

    def create_data_model(self):
        data = {}
        data['distance_matrix'] = self.df.values / 1000  # 转公里
        data['num_vehicles'] = self.num_vehicles
        data['depot'] = self.depot
        return data

    def work(self):
        # step1，初始化数据，得到3个参数的字典
        data = self.create_data_model()

        # 创建路线管理，tsp_size（城市数量）, num_vehicles（车的数量）, depot（原点）
        manager = pywrapcp.RoutingIndexManager(len(data['distance_matrix']),
                                               data['num_vehicles'],
                                               data['depot'])

        # step2，创建 Routing Model.
        routing = pywrapcp.RoutingModel(manager)

        # step3，计算两点之间的距离，输入2个节点的index，输出2个节点之间的距离
        def distance_callback(from_index, to_index):
            # 将index转为distance_matrix中的index
            # Convert from routing variable Index to distance matrix NodeIndex.
            from_node = manager.IndexToNode(from_index)
            to_node = manager.IndexToNode(to_index)
            return data['distance_matrix'][from_node][to_node]

        # step4，注册距离函数
        transit_callback_index = routing.RegisterTransitCallback(
            distance_callback)

        # Define cost of each arc. 定义每个边的成本(arc 指边)
        routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

        # step5，添加距离约束
        dimension_name = 'Distance'
        routing.AddDimension(
            transit_callback_index,
            0,  # no slack 不松弛
            10000,  # 车辆最大行驶距离
            True,  # start cumul to zero 开始累计到零
            dimension_name)
        distance_dimension = routing.GetDimensionOrDie(dimension_name)

        # 尽量减少车辆之间的最大距离 全局差值成本系数100，值大则车辆间距离小；值小，则车辆间距离大；
        distance_dimension.SetGlobalSpanCostCoefficient(100)

        # step6，设置参数策略，Setting first solution heuristic. 设置第一个启发式搜索的策略
        search_parameters = pywrapcp.DefaultRoutingSearchParameters()
        # PATH_CHEAPEST_ARC 指边之间路径最短的策略
        search_parameters.first_solution_strategy = (
            routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC)

        # step7，求解路径规划
        solution = routing.SolveWithParameters(search_parameters)
        # step8，输出结果
        route_list, distance_list = self.get_solution(manager, routing,
                                                      solution)
        return route_list, distance_list


if __name__ == "__main__":
    model = VRP(df_distance, 4, 0)
    route_list, distance_list = model.work()
    print(route_list)
    print(distance_list)

[[0, 7, 18, 19, 17, 22, 6], [0, 21], [0, 5, 20, 23, 24, 26, 27, 25, 12, 32, 11, 31, 30, 13, 16], [0, 14, 28, 29, 10, 9, 8, 15, 1, 4, 2, 3]]
[6341, 7096, 6749, 6845]


In [5]:
sum(distance_list)

27031

# 3 打印每辆车的行车路线

In [11]:
for i, route in enumerate(route_list):
    print(f"第{i + 1}辆车行驶路线为:")
    print(f"由{df_distance.index[route[0]]}出发:-->", end=' ')
    for j in route[1:]:
        if route.index(j) < len(route) - 1:
            print(f"经：{df_distance.index[j]}-->", end=' ')
        else:
            print(f"到：{df_distance.index[j]} 结束")
            print(f"行驶里程为：{distance_list[i]}")
    print("*" * 100)

第1辆车行驶路线为:
由北京出发:--> 经：太原--> 经：银川--> 经：西宁--> 经：兰州--> 经：乌鲁木齐--> 到：呼和浩特 结束
行驶里程为：6341
****************************************************************************************************
第2辆车行驶路线为:
由北京出发:--> 到：拉萨 结束
行驶里程为：7096
****************************************************************************************************
第3辆车行驶路线为:
由北京出发:--> 经：石家庄--> 经：西安--> 经：成都--> 经：重庆--> 经：贵阳--> 经：昆明--> 经：南宁--> 经：海口--> 经：澳门--> 经：广州--> 经：香港--> 经：南昌--> 经：合肥--> 到：济南 结束
行驶里程为：6749
****************************************************************************************************
第4辆车行驶路线为:
由北京出发:--> 经：郑州--> 经：武汉--> 经：长沙--> 经：福州--> 经：杭州--> 经：上海--> 经：南京--> 经：天津--> 经：沈阳--> 经：哈尔滨--> 到：长春 结束
行驶里程为：6845
****************************************************************************************************


# 4 总结：


1. 主要是对这些API比较陌生，要按过程梳理好；官网的步骤很清晰，可以参考。
2. 一辆车时，没有`for vehicle_id in range(self.num_vehicles)`这层循环，solution.ObjectiveValue()=route_distance，但多辆车时，solution.ObjectiveValue()会大于route_distance。
3. 这里我们解决的是，用4辆车一起跑遍33个城市，而不是每一辆车都跑遍33个城市，所以每辆车行驶的路线会不同。